In [179]:
import numpy as np
import pandas as pd

In [180]:
import matplotlib.pyplot as plt
from matplotlib.image import imread

#img = imread('train/0004be2cfeaba1c0361d39e2b000257b.jpg')
#plt.imshow(img)

In [181]:
# use opencv package to load images
import glob
import cv2

images = { file[8:]:cv2.imread(file) for file in glob.glob("./train/*.jpg")}

In [182]:
images['0004be2cfeaba1c0361d39e2b000257b.jpg'].shape
#next(iter(images))

(32, 32, 3)

In [183]:
targets_df = pd.read_csv('train.csv')
targets_df.head()

,id,has_cactus
0,0004be2cfeaba1c0361d39e2b000257b.jpg,1
1,000c8a36845c0208e833c79c1bffedd1.jpg,1
2,000d1e9a533f62e55c289303b072733d.jpg,1
3,0011485b40695e9138e92d0b3fb55128.jpg,1
4,0014d7a11e90b62848904c1418fc8cf2.jpg,1


In [184]:
targets = dict(zip(targets_df.id, targets_df.has_cactus))

In [185]:
labels = []
train = []
for id, image in images.items():
    train.append(image)
    labels.append(targets[id])

labels = np.array(labels)
train = np.array(train)

In [204]:
X_train = train[0:14000]
y_train = labels[0:14000]

X_test = train[14000:]
y_test = labels[14000:]

In [207]:
print("X_train shape is ", X_train.shape)
print("y_train shape is ", y_train.shape)

(14000, 32, 32, 3)

In [256]:
input_shape = (X_train.shape[1], X_train.shape[2], X_train.shape[3])  #(32, 32, 3)
print(input_shape)

(32, 32, 3)


In [210]:
# We now build a CNN for the data set

In [212]:
#import keras
from keras.models import Sequential
from keras.utils import to_categorical
from keras.layers import Dense, Dropout, Conv2D, MaxPooling2D, Flatten

In [249]:
model = Sequential()

model.add(Conv2D(32, kernel_size=(3,3), strides = (1,1), activation = 'relu', input_shape=input_shape))
model.add(MaxPooling2D(pool_size=(2,2),strides=(2,2)))
model.add(Conv2D(64, kernel_size=(3,3), activation='relu'))
model.add(Dropout(0.25))
model.add(MaxPooling2D(pool_size=(2,2), strides=(2,2)))
model.add(Conv2D(64, kernel_size=(3,3), activation='relu'))
#model.add(MaxPooling2D(pool_size=(2,2), strides=(2,2)))
model.add(Flatten())
model.add(Dense(50, activation='relu'))
model.add(Dropout(0.25))
model.add(Dense(2, activation='sigmoid')) #'softmax'))
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_86 (Conv2D)           (None, 30, 30, 32)        896       
_________________________________________________________________
max_pooling2d_75 (MaxPooling (None, 15, 15, 32)        0         
_________________________________________________________________
conv2d_87 (Conv2D)           (None, 13, 13, 64)        18496     
_________________________________________________________________
dropout_3 (Dropout)          (None, 13, 13, 64)        0         
_________________________________________________________________
max_pooling2d_76 (MaxPooling (None, 6, 6, 64)          0         
_________________________________________________________________
conv2d_88 (Conv2D)           (None, 4, 4, 64)          36928     
_________________________________________________________________
flatten_29 (Flatten)         (None, 1024)              0         
__________

In [250]:
from keras import optimizers

model.compile(optimizer=optimizers.RMSprop(lr = 1e-4), #'rmsprop', 
              loss = 'binary_crossentropy', 
              metrics=['accuracy'])

In [251]:
targets=to_categorical(y_train)


In [252]:
X_train_scaled = X_train.astype('float32') / 255.
model.fit(X_train_scaled, targets, epochs = 5)

Epoch 1/5
14000/14000 [==============================] - 11s 813us/step - loss: 0.3460 - acc: 0.8486
Epoch 2/5
14000/14000 [==============================] - 12s 825us/step - loss: 0.1948 - acc: 0.9274
Epoch 3/5
14000/14000 [==============================] - 12s 858us/step - loss: 0.1738 - acc: 0.9367
Epoch 4/5
14000/14000 [==============================] - 12s 858us/step - loss: 0.1573 - acc: 0.9412
Epoch 5/5
14000/14000 [==============================] - 12s 870us/step - loss: 0.1456 - acc: 0.9463


In [255]:
X_test_scaled = X_test.astype('float32') / 255.
score = model.evaluate(X_test_scaled, to_categorical(y_test))
print(score)
print(model.metrics_names)

3500/3500 [==============================] - 1s 285us/step
[0.5652210213116237, 0.7491428571428571]
['loss', 'acc']
